In [3]:
import reacton
import reacton.ipyvuetify as v
import reacton
from reacton import ipyvuetify as iw
from reacton import bqplot
from reacton import ipywidgets as w
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'reacton'

In [ ]:
def initial_irfs():
    t = np.arange(0, 100)
    y1 = np.sin(t)*np.exp(-t/3)
    y2 = np.cos(t)*np.exp(-t/3)
    df1 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

    y1 = np.sin(t)*np.exp(-t/5)
    y2 = np.cos(t)*np.exp(-t/5)
    df2 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

    irfs = {
        'e_y': df1,
        'e_z': df2
    }
    return irfs

irfs0 = initial_irfs()

In [ ]:
# from reacton import matplotlib# # import solara

# def another_plot(irfs, c, selects):


    # # create scales
    # xs = bqplot.LinearScale()
    # ys = bqplot.LinearScale()

    # colors=["blue","red"]
    # # with iw.Card(outlinedd=True,_style="width: 350px; height: 250px"):
    # lines = [
    #     bqplot.Lines(x=irfs[k]['t'], y=irfs[k][c], scales={"x": xs, "y": ys}, labels=[k], colors=colors[i])
    #     for (i,k) in enumerate(irfs.keys()) if k in selects
    # ]
    # # create axes objects
    # xax = bqplot.Axis(scale=xs, grid_lines="solid", label='t')
    # yax = bqplot.Axis(scale=ys, orientation="vertical", label=c, grid_lines="solid")

    # # create the figure object (renders in the output cell)
    # return bqplot.Figure(marks=lines, axes=[xax, yax], legend=True, transition=True)



In [ ]:
from reacton import bqplot
# import solara

def one_plot(irfs, c, selects):

    # create scales
    xs = bqplot.LinearScale()
    ys = bqplot.LinearScale()

    colors=["blue","red"]
    # with iw.Card(outlinedd=True,_style="width: 350px; height: 250px"):
    lines = [
        bqplot.Lines(x=irfs[k]['t'], y=irfs[k][c], scales={"x": xs, "y": ys}, labels=[k], colors=colors[i])
        for (i,k) in enumerate(irfs.keys()) if k in selects
    ]
    # create axes objects
    xax = bqplot.Axis(scale=xs, grid_lines="solid", label='t')
    yax = bqplot.Axis(scale=ys, orientation="vertical", label=c, grid_lines="solid")

    # create the figure object (renders in the output cell)
    with iw.Container() as main:
        bqplot.Figure(marks=lines, axes=[xax, yax], legend=True, transition=True)
    return main



In [ ]:
@reacton.component
def IrfViewer(parameters):
    
    irfs = compute_solution(parameters)

    cols = [str(e) for e in [*irfs.values()][0].columns[1:]]
    n = len(cols)

    ind, set_ind = reacton.use_state(cols[0])

    sel = tuple(irfs.keys())
    selects, set_selects = reacton.use_state( sel )

    with iw.Container() as main:

        with iw.Window(v_model=ind, on_v_model=set_ind, show_arrows=True):
            for (i,c) in enumerate(cols):
                with iw.WindowItem(value=c):
                    # with iw.Card():
                    one_plot(irfs, c, selects)
        
        iw.Select(
            label='Shocks',
            items=sel,
            v_model=selects,
            on_v_model=lambda w: set_selects(w),
            multiple=True,
        )

    return main


In [ ]:
def compute_solution(parameters):
        
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    
    t = np.arange(0, 100)
    y1 = np.sin(t*beta)*np.exp(-t/3*alpha)
    y2 = np.cos(t*beta*alpha)*np.exp(-t/3*alpha)
    df1 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

    y1 = np.sin(t*beta)*np.exp(-t/5*alpha+beta)
    y2 = np.cos(t*alpha*beta)*np.exp(-t/5*alpha-beta)
    df2 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

    irfs = {
        'e_y': df1,
        'e_z': df2
    }

    return irfs

In [ ]:
# @reacton.component
# def ComputeSolution(parameters, set_irf):
    
#     alpha = parameters["alpha"]
#     beta = parameters["beta"]
    
#     t = np.arange(0, 100)
#     y1 = np.sin(t*beta)*np.exp(-t/3*alpha)
#     y2 = np.cos(t*beta*alpha)*np.exp(-t/3*alpha)
#     df1 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

#     y1 = np.sin(t*beta)*np.exp(-t/5*alpha+beta)
#     y2 = np.cos(t*alpha*beta)*np.exp(-t/5*alpha-beta)
#     df2 = pd.DataFrame({'t': t, 'y1': y1, 'y2': y2})

#     irfs = {
#         'e_y': df1,
#         'e_z': df2
#     }

#     set_irf(irfs)
    
#     with iw.Container() as main:
#         iw.Text(children=['Nothing'])
    
#     return main


In [ ]:

@reacton.component
def ParameterChooser(args, parameters, set_parameters):
    parms = {k: reacton.use_state(v) for k,v in parameters.items()}
    set_parameters({k: parms[k][0] for k in parameters.keys()})
    # with iw.Card(level=2):
        # iw.Label(children=["Parameters"])
    with iw.Container() as main:
        for k, (v, set_v) in parms.items():
            iw.Slider(
                v_model=v,
                on_v_model=set_v,
                label=k,
                min=args[k][1],
                max=args[k][2],
                step=0.01,
                thumb_label=True,
                dense=False,
                hide_details=True,
            )
    
    return main

In [ ]:

@reacton.component
def Page():

    # initial guesses
    args = {
        "alpha": [0.4, 0.0, 1.0], 
        "beta": [0.5, 0.0, 10]
    }

    irfs0 = initial_irfs()


    parameters, set_parameters = reacton.use_state({k:v[0] for k,v in args.items()})
    
    # irf, set_irf = reacton.use_state(irfs0)

    # ComputeSolution(parameters, set_irf) # component: updates irf when parameters change
    
    with iw.ExpansionPanels(multiple=True) as main:
        with iw.ExpansionPanel():
            iw.ExpansionPanelHeader(children="Simulation")
            with iw.ExpansionPanelContent():
                # IrfViewer(irf) # visualize irf
                IrfViewer(parameters)

        with iw.ExpansionPanel():
            iw.ExpansionPanelHeader(children="Parameters")
            with iw.ExpansionPanelContent():
                ParameterChooser(args, parameters, set_parameters)  # component: ui to choose parameters
            # this is for ipyvuetify>=3
            # with iw.ExpansionPanelText():
                # ParameterChooser(args, parameters, set_parameters)
    
    # set_irf(compute_solution(parameters))

    return main

In [ ]:
Page()